# Sandbox code for tiling the BGS/GAMA fields

Below is some simple code to tile the GAMA fields, but it needs more work to ensure 4-pass coverage of most targets.

## Tile the G02, G09, G12, and G15 GAMA fields.

As an initial effort, we tile each of the GAMA fields very simplistically, using constant offsets in RA and Dec.  The next iteration of this notebook will need to consider the dither pattern required to fill in the gaps in the DESI focal plane due to the central bullseye (0.2 deg) and the GFA gaps (0.4 deg).

In [ ]:
def simple_radec_off_scheme(ras, decs, dx=0.6, ang=42):
    """Code written by E. Schlafly (LBNL).
    
    Take a single covering and define four sets of offsets, starting with something 
    minimal. 
    
    Need to cover central bulls-eyes (0.2 deg) and GFA gaps (p to 0.4 deg).

    """
    ang = np.radians(ang)
    dang = np.pi/2
    dithers = [[0, 0], 
               [dx*np.sin(ang+0*dang), dx*np.cos(ang+0*dang)], 
               [dx*np.sin(ang+1*dang), dx*np.cos(ang+1*dang)],
               [dx*np.sin(ang+2*dang), dx*np.cos(ang+2*dang)]]
    dithers = np.cumsum(np.array(dithers), axis=0)
    dithers = list(dithers) + [[np.mean([d[0] for d in dithers]),
                                np.mean([d[1] for d in dithers])]]
    fac = 1./np.cos(np.radians(decs))
    fac = np.clip(fac, 1, 360*5)  # confusion near celestial pole.
    newras = np.concatenate([ras+d[0]*fac for d in dithers])
    newdecs = np.concatenate([decs+d[1] for d in dithers])
    newdecs = np.clip(newdecs, -np.inf, 90.)
    newras = newras % 360
    newras = np.concatenate([newras, newras])
    newdecs = np.concatenate([newdecs, newdecs])
    
    return newras, newdecs

In [ ]:
def radec2tiles(ra, dec, tileid_start=100000):
    """Given an input set of tile coordinates (centers), return a 
    table of tiles.
    
    """
    from astropy.table import Table, Column
    
    ntile = len(np.atleast_1d(ra))
    
    tiles = Table()
    tiles.add_column(Column(name='TILEID', dtype='i4', length=ntile))
    tiles.add_column(Column(name='RA', dtype='f8', length=ntile))
    tiles.add_column(Column(name='DEC', dtype='f8', length=ntile))
    tiles.add_column(Column(name='PASS', dtype=np.int16, length=ntile))
    tiles.add_column(Column(name='IN_DESI', dtype=np.int16, length=ntile))
    tiles.add_column(Column(name='EBV_MED', dtype='f4', length=ntile))
    tiles.add_column(Column(name='AIRMASS', dtype='f4', length=ntile))
    tiles.add_column(Column(name='STAR_DENSITY', dtype='f4', length=ntile))
    tiles.add_column(Column(name='EXPOSEFAC', dtype='f4', length=ntile))
    tiles.add_column(Column(name='PROGRAM', dtype='S6', length=ntile))
    tiles.add_column(Column(name='OBSCONDITIONS', dtype='i4', length=ntile))
    
    tiles['RA'] = ra
    tiles['DEC'] = dec
    tiles['TILEID'] = (np.arange(ntile) + tileid_start).astype('i4')
    tiles['PASS'] = 1          # ??
    tiles['AIRMASS'] = 1.0     # to be filled in by surveysim
    tiles['IN_DESI'] = 1       # assume all "in DESI"
    tiles['EXPOSEFAC'] = 1.0   # ??
    tiles['PROGRAM'] = 'DARK'
    tiles['OBSCONDITIONS'] = 1 # ??
    
    return tiles

In [ ]:
def plot_tile(ra, dec, r=1.606, color='k', ax=None):
    '''Approximate plot of tile location'''
    ang = np.linspace(0, 2*np.pi, 100)
    x = ra + r*np.cos(ang)/np.cos(np.radians(dec))
    y = dec + r*np.sin(ang)
    if ax is None:
        fig, ax = plt.subplots()
    ax.plot(x,y, '-', color=color, alpha=0.5)

In [ ]:
def read_gama_tiles(gamafield=None):
    if gamafield:
        _tilesfile = tilesfile.replace('.fits', '-{}.fits'.format(gamafield))
    else:
        _tilesfile = tilesfile
    
    if os.path.isfile(_tilesfile):
        tiles = Table(fitsio.read(_tilesfile, ext=1))
        print('Read {} tiles from {}'.format(len(tiles), _tilesfile))
    else:
        print('Tiles file {} not found!'.format(_tilesfile))
        tiles = []
    return tiles

In [ ]:
def qa_gama_tiles(gama=None, tiles=None, gamafield=None, overwrite=False):
    
    if gama is None:
        gama = read_gama_sample()
    if tiles is None:
        tiles = read_gama_tiles(gamafield=gamafield)

    fig, ax = plt.subplots(2, 2, figsize=(10, 6))
    ax = ax.reshape(4)
    
    for thisax, field in zip(ax, sorted(set(gama['FIELD']))):
        
        infield = gama['FIELD'] == field
    
        if np.count_nonzero(infield) > 0:
            thisax.scatter(gama['RA'][infield], gama['DEC'][infield], 
                           s=1, alpha=0.5, marker='s')
        
            ww = ( (tiles['RA'] > gama['RA'][infield].min()) * (tiles['RA'] < gama['RA'][infield].max()) * 
                   (tiles['DEC'] > gama['DEC'][infield].min()) * (tiles['DEC'] < gama['DEC'][infield].max()) )
            for tt in tiles[ww]:
                plot_tile(tt['RA'], tt['DEC'], ax=thisax)
            
        thisax.set_xlabel('RA')
        thisax.set_ylabel('Dec')
        thisax.invert_xaxis()
        thisax.set_title(field)
    
    fig.tight_layout()
    if overwrite:
        pngfile = os.path.join(basedir, 'qaplots', 'qa-gama-tiles.png')
        print('Writing {}'.format(pngfile))
        fig.savefig(pngfile)    

In [ ]:
def gama_tiles(gama, radius=1.6, gamafield=None, debug=False, overwrite=False):
    """Tile each of the GAMA fields."""
    
    gama_radec = { # Field corners
        'G02': (30.2, 38.8, -6, -4),
        'G09': (129, 141, -2, 3),
        'G12': (174, 186, -3, 2),
        'G15': (211.5, 223.5, -2, 3),
        'G23': (339, 351, -35, -30)
    }    

    if gamafield:
        keep = [ff.decode('utf-8') == gamafield for ff in gama['FIELD'].data]
        gama = gama[keep]
        print(gama['RA'].min(), gama['RA'].max(), gama['DEC'].min(), gama['DEC'].max())
    
        _tilesfile = tilesfile.replace('.fits', '-{}.fits'.format(gamafield))
    else:
        _tilesfile = tilesfile
        
    if not overwrite and os.path.isfile(_tilesfile):
        tiles = Table.read(_tilesfile)
        print('Read {} tiles from existing file {}'.format(len(tiles), _tilesfile))
    
    ra_allfields, dec_allfields = [], []
    for field in sorted(set(gama['FIELD'])):
        infield = gama['FIELD'] == field

        # Get the tile centers very simplistically.
        nra = np.ceil( (gama_radec[field][1] - gama_radec[field][0]) / radius ).astype('int') + 1
        ndec = np.ceil( (gama_radec[field][3] - gama_radec[field][2]) / radius ).astype('int') - 1

        allra = np.linspace(gama_radec[field][0] - radius/2, gama_radec[field][1] + radius/2, nra)
        if ndec == 1:
            alldec = np.repeat( np.mean( [gama_radec[field][2], gama_radec[field][3]] ), ndec)
        else:
            alldec = np.linspace(gama_radec[field][2] + radius/2, gama_radec[field][3] - radius/2, ndec)

        ra, dec = [], []
        for dd in alldec:
            ra.append(allra)
            dec.append(np.repeat(dd, nra))

        ra = np.array(ra).flatten()
        dec = np.array(dec).flatten()

        # Simpler dither scheme
        #ra2, dec2 = simple_radec_off_scheme(ra, dec, dx=1.4, ang=330)
        #ra2, dec2 = simple_radec_off_scheme(ra, dec, dx=0.6, ang=42)

        ra_allfields.append(ra)
        dec_allfields.append(dec)
        
        if debug:
            fig, ax = plt.subplots()
            ax.scatter(gama['RA'][infield], gama['DEC'][infield], s=1, alpha=0.5)
            ax.invert_xaxis()
            ax.title(field)
            for rr, dd in zip(ra, dec):
                plot_tile(rr, dd, ax=ax)
            plt.show()

    ra = np.hstack(ra_allfields).flatten()
    dec = np.hstack(dec_allfields).flatten()
            
    tiles = radec2tiles(ra, dec)
    if gamafield == 'G02':
        print('Temporary hack -- writing just the first two tiles!')
        tiles = tiles[:2]
    
    if overwrite:
        print('Writing {}'.format(_tilesfile))
        tiles.write(_tilesfile, overwrite=overwrite)
    
    return tiles

In [ ]:
tiles = gama_tiles(gama, debug=False, overwrite=overwrite_tiles)
tiles

In [ ]:
qa_gama_tiles(gama, tiles, overwrite=overwrite_tiles)

#### Read/write a tile file of just the G02 field.

In [ ]:
G02tiles = gama_tiles(gama, gamafield='G02', debug=False, overwrite=overwrite_tiles)
G02tiles